## 1. 멀티스레딩 

- threading 모듈을 이용하여 스레드를 생성하고 관리할 수 있습니다. 
- 그러나 파이썬의 Global Interpreter Lock (GIL)이라는 특징 때문에 CPU-bound 작업에서는 스레드가 진정한 병렬 처리를 제공하지 않을 수 있습니다. 
- 그러나 I/O-bound 작업에서는 스레드가 도움이 될 수 있습니다.

In [1]:
import threading

def print_numbers():
    for i in range(5):
        print(i)

def print_letters():
    for letter in 'ABCDE':
        print(letter)

thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_letters)

thread1.start()
thread2.start()

thread1.join()
thread2.join()


0
1
2
3
4
A
B
C
D
E


## 2. 멀티프로세싱(Multiprocessing): 

- 멀티프로세싱은 별도의 프로세스를 생성하여 동시성을 달성하는 방식입니다. 
- 각 프로세스는 독립적으로 실행되므로 GIL의 영향을 받지 않습니다. 
- multiprocessing 모듈을 사용하여 멀티프로세싱을 구현할 수 있습니다.

In [2]:
from multiprocessing import Process

def print_numbers():
    for i in range(5):
        print(i)

def print_letters():
    for letter in 'ABCDE':
        print(letter)

process1 = Process(target=print_numbers)
process2 = Process(target=print_letters)

process1.start()
process2.start()

process1.join()
process2.join()


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/a06411/opt/anaconda3/envs/fluentPython/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/a06411/opt/anaconda3/envs/fluentPython/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/a06411/opt/anaconda3/envs/fluentPython/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/a06411/opt/anaconda3/envs/fluentPython/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'print_letters' on <module '__main__' (built-in)>
    self = reduction.pickle.load(fro

## 충돌 해결 

- 문제의 원인은 multiprocessing 모듈을 사용할 때, 메인 모듈에 정의된 함수를 실행하는 것이 어렵다는 점입니다. 
- 이는 함수가 모듈로서 임포트되었을 때, multiprocessing에서 파이썬 인터프리터의 __main__ 모듈로 임포트되기 때문입니다.

- 이 문제를 해결하기 위해서는, 실행할 함수들을 별도의 모듈로 분리하고 그 모듈을 multiprocessing 모듈에서 사용하는 것이 일반적인 방법입니다.

In [4]:
%%writefile worker.py
# worker.py

def print_numbers():
    for i in range(5):
        print(i)

def print_letters():
    for letter in 'ABCDE':
        print(letter)

Writing worker.py


In [5]:
from multiprocessing import Process
from worker import print_numbers, print_letters

if __name__ == '__main__':
    process1 = Process(target=print_numbers)
    process2 = Process(target=print_letters)

    process1.start()
    process2.start()

    process1.join()
    process2.join()

A
B
C
D
E
0
1
2
3
4


## 3. 비동기 프로그래밍(Asynchronous Programming): 

- asyncio 모듈을 사용하여 비동기적으로 작업을 수행할 수 있습니다. 
- 비동기 프로그래밍은 I/O-bound 작업에 특히 유용하며, 이벤트 루프를 통해 비동기 코드를 실행합니다.

In [8]:
import platform

In [9]:
platform.python_version()

'3.11.3'

In [2]:
import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

await main()

Hello ...
... World!


In [4]:
import asyncio

async def print_numbers():
    for i in range(5):
        print(i)
        await asyncio.sleep(1)

async def print_letters():
    for letter in 'ABCDE':
        print(letter)
        await asyncio.sleep(1)
        
        
async def main():

    await print_numbers()
    await asyncio.sleep(1)
    await print_letters()
    
await main()


0
1
2
3
4
A
B
C
D
E


## 해결방안

In [9]:
import asyncio

async def print_numbers():
    for i in range(5):
        print(i)
        await asyncio.sleep(1)

async def print_letters():
    for letter in 'ABCDE':
        print(letter)
        await asyncio.sleep(1)

async def main():
    task1 = asyncio.create_task(print_numbers())
    task2 = asyncio.create_task(print_letters())

    await asyncio.gather(task1, task2)


await main()


0
A
1
B
2
C
3
D
4
E
